# Splitting the integers 1 to 36

[Link](https://puzzling.stackexchange.com/questions/115935/splitting-the-integers-1-to-36) to the original problem on StackExchange.

Split the integers 1 to 36 into two sets, A and B, such that any number in set A has a common divisor greater than 1 with no more than two other numbers in A, but for every number in B there are at least three numbers in A with which it has a common divisor.

How large can set A be?

In general, for which N is such a splitting of the integers 1 to N possible?

In [1]:
#!pip install ortools
from ortools.sat.python import cp_model

In [2]:
import itertools
import math
import numpy as np

In [3]:
N = 36

# Creates the model and set solver
model = cp_model.CpModel()
solver = cp_model.CpSolver()

# Bool variables for sets A and B
A = {(i): model.NewBoolVar(f'A_{i}') for i in range(1,N+1)}
B = {(i): model.NewBoolVar(f'B_{i}') for i in range(1,N+1)}
# Int variable for int series
z = {(i): model.NewIntVar(1, N, f'z_{i}') for i in range(1,N+1)}
# Storing max, min and modulo values
mins = {(i): model.NewIntVar(1,N, f'mins_{i}') for i in range(1,math.perm(N,2)+1)}
maxs = {(i): model.NewIntVar(1,N, f'maxs_{i}') for i in range(1,math.perm(N,2)+1)}
mods = {(i,j): model.NewIntVar(0,N, f'mods_{i}_{j}') for i,j in list(itertools.permutations(z.keys(),2))}
# Storing shared divisors
share_div = {(i,j): model.NewBoolVar(f'share_div_{i}_{j}') \
             for i,j in list(itertools.permutations(z.keys(),2))}
# Boolean between sets A and B
boolVarA = {(i,j): model.NewBoolVar(f'boolVarA_{i}_{j}') for i,j in list(itertools.permutations(z.keys(),2))}


# Each number only appears once
model.AddAllDifferent([z[i] for i in z.keys()])

for i in range(1,N+1):
    # Numbers in ascending order
    if i < N:
        model.Add(z[i] <= z[i+1])
    # Each number must be used only once between sets
    model.Add(A[i]+B[i] == 1)

count = 1
for i,j in list(itertools.permutations(z.keys(),2)):
    # Grabbing min and max between permutations and getting their modulo value
    model.AddMinEquality(mins[count], [z[i], z[j]])
    model.AddMaxEquality(maxs[count], [z[i], z[j]])
    model.AddModuloEquality(mods[i,j], maxs[count], mins[count])

    # share_div stores 1 if modulo of 2 nums is 0, 0 otherwise
    model.Add((mods[i,j] == 0)).OnlyEnforceIf(share_div[i,j])
    model.Add((mods[i,j] != 0)).OnlyEnforceIf(~share_div[i,j])

    # boolVarA stores 1 if A[i] and A[j] are in set A and share a divisor
    model.AddMultiplicationEquality(boolVarA[i,j], [A[i],A[j], share_div[i,j]])
    count+=1

for i in range(1,N+1):
    model.Add(sum(boolVarA[i,j] for j in range(1,N+1) \
                  if (i,j) in boolVarA.keys() and 1 not in [i,j])<=2)
    

model.Maximize(sum(A[i] for i in A.keys()))
status = solver.Solve(model)

print(f"Status = {solver.StatusName(status)}")

if solver.StatusName(status) == 'OPTIMAL':
    print(f'Solution = {sum(solver.value(A[i]) for i in A.keys())}')
    sol_A = np.array([solver.value(A[i])*solver.value(z[i]) for i in A.keys() if solver.value(A[i]) == 1])
    sol_B = np.array([solver.value(B[i])*solver.value(z[i]) for i in B.keys() if solver.value(B[i]) == 1])
    print(f'Set A = {sol_A}')
    print(f'Set B = {sol_B}')

Status = OPTIMAL
Solution = 28
Set A = [ 1  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
 31 33 34 35]
Set B = [ 2  3  4  5  6  7 32 36]


In [4]:
# Num of divisors for each item in set A (within sets)

x = []
for i in sol_A:
    x.append(sum([max(i, j) % min(i, j) == 0 for j in sol_A if i != j and 1 not in [i,j]]))

np.array(x)

array([0, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 2, 0, 1, 1, 1,
       0, 2, 0, 1, 1, 0])

In [5]:
# Num of divisors for each item in set B (against set A)

y = []
for i in sol_B:
    y.append(sum([max(i, j) % min(i, j) == 0 for j in sol_A]))

np.array(y)

array([14, 10,  7,  7,  5,  5,  3,  4])